# Imports

## Standard Library Modules

In [ ]:
from datetime import datetime
import csv

## Third Party Modules

## Constants

### Global Constants

In [ ]:
import GlobalConstants

### Debug Level Flags

In [ ]:
#import DebugConstants as db

# Stuff to be Moved

## Testing

In [ ]:
# def test_instance_generation(num_tests):
#     assert not is_connected(nx.empty_graph(NUM_NODES))
#     assert is_connected(get_random_connected_graph())
#     G = get_random_connected_graph()
#     for i in range(num_tests):
#         request = MulticastRequest(MAX_MULTICAST_SIZE, G)
#         source = request.source
#         recipients = request.recipients
#         assert request.source is not None
#         assert request.recipients
#         assert not request.source in request.recipients
#         assert isinstance(request.recipients, set)
#     instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
#     for request in instance.requests:
#         assert request.source in instance.graph
#         assert request.recipients.issubset(instance.graph)

# def test_LP_creation(num_tests):
#     instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
#     reduced_LP = create_LP(instance.graph, instance.requests)
#     assert len(reduced_LP.getVars()) == 1
#     assert len(reduced_LP.getConstrs()) == NUM_EDGES + NUM_MULTICAST_REQUESTS, "Num of constraints: {} Expected: {}".format(len(reduced_LP.getConstrs()), NUM_EDGES + NUM_MULTICAST_REQUESTS)
    
# def test_subproblem(num_tests):
#     instance = MulticastPackingInstance(NUM_MULTICAST_REQUESTS, MAX_MULTICAST_SIZE)
#     reduced_LP = create_LP(instance.graph, instance.requests)
#     column_generator = Approx2MulticastPackingColumnGenerator(instance, reduced_LP)
    
#     new_trees = column_generator.generate_new_trees()
#     assert (len(new_trees) == instance.num_requests)
#     for tree in new_trees:
#         assert cost(tree, 1) == len(tree.edges()), "cost: {} edges: {}".format(cost(tree, 1), len(tree.edges()))
           
#     assert len(reduced_LP.getVars()) == 1+instance.num_requests
    
# def run_tests(num_tests):
#     test_instance_generation(num_tests)
#     test_LP_creation(num_tests)
#     test_subproblem(num_tests)

# NUM_TESTS = 0
# #run_tests(NUM_TESTS)

## Rounding Algorithms

### (Totally) Randomized Rounding

In [ ]:
def randRoundingHelper(fracSoln):
    randNum = random.random()
    for T in fracSoln:
        randNum -= fracSoln[T]
        if randNum < 0:
            return T
    
    
def randRounding(MCPsolver):
    intSoln = list()
    fracSoln = MCPsolver.solution[MCPsolver.iteration-1]
    for i in range(MCPsolver.instance.num_requests):
        intSoln.append(FrozenDict({randRoundingHelper(fracSoln[i]) : 1}))
        
    return tuple(intSoln)

# Run the solver

In [ ]:
from RunExpirement import runExpirement
datetime_str = datetime.now().strftime("%Y-%m-%d_%H_%M")
print(datetime_str)
tableHeaders = ["Label",
                "Solver", 
                "Block Apx Ratio", 
                "Vertices",
                "Edges",
                "Requests",
                "Group Size", 
                "Max Delay",
                "Congestion", 
                "Potential",
                "Iterations",
                "Total Time",
                "Avg. Time/It",
                "Stop Flags"
                ]
with open("outputs/{}.csv".format(datetime_str),'w') as allFile:
        allWriter = csv.writer(allFile)
        allWriter.writerow(tableHeaders)
        
# with open("outputs/summaries/summary_{}.csv".format(datetime_str),'w') as sumFile:
#         sumWriter = csv.writer(sumFile)
#         sumWriter.writerow(tableHeaders)

2022-07-27_13_25


In [ ]:
paramList = []

paramList += [("Difficulty by Size",
               n,      # Nodes
               d*n//2, # Edges
               k,      # Requests
               s,      # Group Size
               0       # Delay
              )
              for k in range(1,2)
              for n in range(24, 50, 2) 
              for d in range(3, n)
              for s in range(2, n+1)
             ]

numReps = 1

solverList = []
runExpirement(datetime_str, numReps, paramList, [2], ["WS"], solverList)

Parameter Loop: |V|=24, |E|=36, k=1, |S_i|<=2, D = 0
	 Instance Repitition: 0
		 Algo: WS 	 Block Approx: 2
4
01100
lambda(x): 0.5
phi_t(x): -0.6931471805599453
tolerance: 0.0

Parameter Loop: |V|=24, |E|=36, k=1, |S_i|<=3, D = 0
	 Instance Repitition: 0
		 Algo: WS 	 Block Approx: 2
22
01100
lambda(x): 0.33333333333333326
phi_t(x): -1.09861228866811
tolerance: -8.32667268468867e-17

Parameter Loop: |V|=24, |E|=36, k=1, |S_i|<=4, D = 0
	 Instance Repitition: 0
		 Algo: WS 	 Block Approx: 2
93
01100
lambda(x): 0.3333333333333333
phi_t(x): -1.0986122886681098
tolerance: -1.66533453693773e-16

Parameter Loop: |V|=24, |E|=36, k=1, |S_i|<=5, D = 0
	 Instance Repitition: 0
		 Algo: WS 	 Block Approx: 2
39
01100
lambda(x): 0.4
phi_t(x): -0.916290731874155
tolerance: 6.93889390390723e-17

Parameter Loop: |V|=24, |E|=36, k=1, |S_i|<=6, D = 0
	 Instance Repitition: 0
		 Algo: WS 	 Block Approx: 2
87
01100
lambda(x): 0.39999999999999997
phi_t(x): -0.9162907318741551
tolerance: 0.0

Parameter Loop

# End

In [ ]:
for solver in solverList:
    print(type(solver))
    print(solver.t)
    print("{:05b}".format(solver.stop_flag))

<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
<class 'Solvers.Impls.WarmStartColGenMcpSolver.WarmStartColGenMcpSolver'>
None
01100
